<a href="https://colab.research.google.com/github/nim17/Flipkart-mobile-price-/blob/master/GPT_pos_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir
mypath="//content/test_data"
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#onlyfiles = [f for f in onlyfiles if f.split('.')[-1]=='htm']
onlyfiles

['Topic_Four.htm',
 'Flexible_Working.htm',
 'New_Topic1.htm',
 'Topic_Three.htm',
 'source_control.htm',
 'First_Topic.htm',
 'Topic_One.htm',
 'Healthcare.htm',
 'homepagefooter.htm',
 'toc.htm',
 'Cars.htm',
 'command_line.htm',
 'contents_panel.htm',
 'outputs_frameless.htm',
 'outputs_general.htm',
 'New_Topic.htm',
 'dropdowns.htm',
 'publishing.htm',
 'Equal_Opportunities.htm',
 'Expenses.htm',
 'Attendance.htm',
 'upgrading_to_2019.htm',
 'Bookings.htm',
 'find_and_replace.htm',
 'Harassment.htm',
 'reviewing.htm',
 'bits_and_pieces.htm',
 'outputs_mobile_apps.htm',
 'outputs_word.htm',
 'images_and_videos.htm',
 'Dependants_Leave.htm',
 'Attendance_e_Book.htm',
 'merged_help.htm',
 'snippets.htm',
 'Health_and_Safety.htm',
 'browse_sequences.htm',
 'Disciplinary_Procedure.htm',
 'html5.skn',
 'outputs_chm.htm',
 'topicfooter.htm',
 'working_on_topics.htm',
 'search.htm',
 'topic.htm',
 'Default_Topic.htm',
 'outputs_pdf.htm',
 'outputs_xml.htm',
 'importing_linking.htm',
 'cal

In [19]:
path="/content/test_data/"

f=open(f'{path}Attendance.htm')
from bs4 import BeautifulSoup
"""soup=BeautifulSoup(text)
for hit in soup.findAll('body'):
    print (hit.text)"""
#print(text)
only_files=onlyfiles[3:]
text_list=[]
i=0
for t in onlyfiles:
    p=path+t
   # print(p)
    f = open(f"{p}", "r")
    try:
        text=f.read()
        soup=BeautifulSoup(text)
        text=[hit.text for hit in soup.findAll('body')]
        text="".join(text)
        text_list.append(text)
    except UnicodeDecodeError:
        i+=1
        print(i)    
text_list[0]

1


'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTopic Heading Three\nSection Heading\nThis is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text.\nThis is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text.\nThis is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text. This is some sample text.\nSection Heading\nThis is some sample text. This is some 

In [0]:
def prep(text):
    esc=['\n','\r','\t','\b','\f','\ooo','\h',"\'"]
    text=text.replace("\n"," ")
    text=[t for t in text if t not in esc]
    text=''.join(text)
    text=" ".join(text.split())
    text=text.replace('Â ', '')
    text=text.split(".")[2:]
    text=". ".join(text)
    return text
text=[prep(t) for t in text_list]

In [21]:
text_list=text.copy()
len(text_list)

76

In [22]:
import torch
from transformers import OpenAIGPTTokenizer, OpenAIGPTLMHeadModel
import time
from matplotlib import pyplot as plt
total_matches=0
total_length=0
accuracy_list=[]
times_list=[]
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
# import logging
# logging.basicConfig(level=logging.INFO)
# Load pre-trained model tokenizer (vocabulary)
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
# Load pre-trained model (weights)
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
# Encode a text inputs
# text = "Who was Jim Henson? Jim Henson was a man"
# text = "Delete this text and replace it with your"
right_confidence=[]
wrong_confidence=[]
correct_words=[]
for t in text_list:
    text = t
    indexed_tokens = tokenizer.encode(text)


    # Convert indexed tokens in a PyTorch tensor
    tokens_tensor = torch.tensor([indexed_tokens])
    input_tensor = tokens_tensor[:,:-1]
    output_tensor = tokens_tensor[:,1:]
    # print("input,output: ",tokens_tensor, input_tensor)
    

    # Set the model in evaluation mode to deactivate the DropOut modules
    # This is IMPORTANT to have reproducible results during evaluation!
    model.eval()
    matches = 0
    no_match=0
    length=0
    # If you have a GPU, put everything on cuda
    #tokens_tensor = tokens_tensor.to('cuda')
    #model.to('cuda')
    length=len(input_tensor[0])
    print("Sentence Length = ",length)
        
    times_array = []
    i=2
    while(i<len(input_tensor[0])+1):
    #for i in range(2,len(input_tensor[0])+1):
        if i>=len(input_tensor[0]):
              break
        token=tokenizer.decode(input_tensor[0][i].reshape(1,1))
        if token=='.':
            i+=4
            if i>=len(input_tensor[0]):
              break
            continue
        start_time = time.time()
        # Predict all tokens
        with torch.no_grad():
    #         print("input shape", input_tensor[0][:i].shape)
            if(i<100):
                outputs = model(input_tensor[:,:i])
            else:
                outputs = model(input_tensor[:,i-100:i])
            predictions = outputs[0]
    #         print("output shape", predictions.shape)
            predicted_index = torch.argmax(predictions[0, -1, :]).item()
    #         print("predicted index: ", predicted_index)
            confidence=torch.max(predictions[0, -1, :]).item()
            if(predicted_index == tokens_tensor[0][i]):
                matches+=1
                right_confidence.append(confidence)
                correct_words.append(tokenizer.decode([predicted_index]))
            else:
                wrong_confidence.append(confidence)
                no_match+=1
                #predicted_text = tokenizer.decode([predicted_index])
                #print('confidence= ',confidence)
                #print("predicted text", predicted_text)
            elapsed = time.time() - start_time
            times_array.append(elapsed)
            predicted_text = tokenizer.decode([predicted_index])
        print(i,end=" ")
        i+=1
    #         print(i,"given: ",tokenizer.decode(list(input_tensor[0][:i])), ", predicted: ", predicted_text, ", time: ", elapsed)
    total_matches+=matches
    total_length+=no_match+matches
    if matches+no_match >0:
      accuracy=matches/(matches+no_match)
      print("matches= ",matches,"\taccuracy=",accuracy)
      accuracy_list.append(accuracy)
    times_list.append(times_array)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


Sentence Length =  442
2 3 4 9 10 15 16 21 22 27 28 33 34 39 40 45 46 51 52 57 58 63 64 69 70 75 76 81 82 87 88 93 94 99 100 105 106 111 112 117 118 123 124 129 130 135 136 141 142 147 148 153 154 159 160 165 166 171 172 177 178 183 184 189 190 191 192 197 198 203 204 209 210 215 216 221 222 227 228 233 234 239 240 245 246 251 252 257 258 263 264 269 270 275 276 281 282 287 288 293 294 299 300 305 306 311 312 317 318 323 324 329 330 335 336 341 342 347 348 353 354 359 360 365 366 371 372 377 378 383 384 389 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411 416 417 418 419 420 421 422 427 428 429 430 431 432 437 438 439 440 

Token indices sequence length is longer than the specified maximum sequence length for this model (1804 > 512). Running this sequence through the model will result in indexing errors


441 matches=  126 	accuracy= 0.7325581395348837
Sentence Length =  1803
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 56 57 58 59 60 61 62 63 64 65 66 67 68 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 28

Token indices sequence length is longer than the specified maximum sequence length for this model (1169 > 512). Running this sequence through the model will result in indexing errors


...

303 matches=  52 	accuracy= 0.21138211382113822
Sentence Length =  732
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 94 95 96 97 98 99 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 210 211 212 213 214 215 216 217 222 223 224 225 226 227 228 229 230 231 232 233 234 235 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308

Token indices sequence length is longer than the specified maximum sequence length for this model (1031 > 512). Running this sequence through the model will result in indexing errors


465 matches=  88 	accuracy= 0.24175824175824176
Sentence Length =  1030
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 154 155 156 157 158 159 160 161 162 163 164 165 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 210 211 212 213 214 215 216 221 222 223 224 225 226 227 228 229 230 231 232 233 234 239 240 241 242 243 244 249 250 251 252 253 254 259 260 261 262 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 310 311 312 313 314 315 31

Token indices sequence length is longer than the specified maximum sequence length for this model (1091 > 512). Running this sequence through the model will result in indexing errors


413 matches=  85 	accuracy= 0.25
Sentence Length =  1090
2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 195 196 197 198 199 200 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 2

In [23]:
def averageOfList(num):
    sumOfNumbers = 0
    for t in num:
        sumOfNumbers = sumOfNumbers + t

    avg = sumOfNumbers / len(num)
    return avg
averageOfList(accuracy_list)

0.301989555130013

In [27]:
max(accuracy_list)

0.7383720930232558

In [0]:
MyFile=open('accuracies_gpt.txt','w')
i=0
for element in [str(t) for t in accuracy_list]:
    try:
        MyFile.write(element)
        MyFile.write('\n')
    except:
        i+=1
        print(i)
MyFile.close()

In [0]:
from google.colab import files
files.download('accuracies_gpt.txt')

In [0]:
import spacy
nlp=spacy.load('en_core_web_sm')
doc=nlp(" ".join(correct_words))

In [43]:
[[token.text,token.pos_] for token in doc if token.text!=" " and token.text!='text' and token.text!='sample' ]

[['"', 'PUNCT'],
 ['=', 'SYM'],
 ['"', 'PUNCT'],
 ['the', 'DET'],
 ['employees', 'NOUN'],
 ['in', 'ADP'],
 ['lives', 'NOUN'],
 ['to', 'ADP'],
 ['and', 'CCONJ'],
 ['to', 'ADP'],
 ['for', 'ADP'],
 [',', 'PUNCT'],
 ['the', 'DET'],
 ['company', 'NOUN'],
 ['be', 'AUX'],
 ['ably', 'ADV'],
 ['or', 'CCONJ'],
 ['the', 'DET'],
 ['is', 'AUX'],
 ['is', 'AUX'],
 ['to', 'PART'],
 ['be', 'AUX'],
 [',', 'PUNCT'],
 ['the', 'DET'],
 ['company', 'NOUN'],
 ['eves', 'VERB'],
 ['its', 'DET'],
 ['as', 'SCONJ'],
 ['possible', 'ADJ'],
 ['be', 'AUX'],
 ['they', 'PRON'],
 ['conditions', 'NOUN'],
 ['is', 'AUX'],
 ['of', 'ADP'],
 ['conditions', 'NOUN'],
 ['employment', 'VERB'],
 ['to', 'ADP'],
 ['for', 'ADP'],
 ['is', 'AUX'],
 ['to', 'PART'],
 ['to', 'ADP'],
 ['the', 'DET'],
 ['working', 'NOUN'],
 ['for', 'ADP'],
 ['who', 'PRON'],
 ['are', 'AUX'],
 ['in', 'ADP'],
 ['a', 'DET'],
 ['request', 'NOUN'],
 ['the', 'DET'],
 ['of', 'NOUN'],
 ['is', 'AUX'],
 ['the', 'DET'],
 ['the', 'DET'],
 ['application', 'NOUN'],
 ['is'